In [3]:
import jax
import jax.numpy as jnp
import flax.linen as nn
import numpy as np
import optax
from flax.linen.initializers import constant, orthogonal
from typing import Sequence, NamedTuple, Any
from flax.training.train_state import TrainState
import distrax
import gymnax
from gymnax.wrappers.purerl import  FlattenObservationWrapper
import wandb


# Custom TrainState to track step count
class CustomTrainState(TrainState):
    step_count: int


from wrappers import (
    LogWrapper,
    OptimisticResetVecEnvWrapper,
    AutoResetEnvWrapper,
    BatchEnvWrapper,
    ThinkingWrapper,
)


ModuleNotFoundError: No module named 'distutils'

In [2]:
config["NUM_UPDATES"] = (
    config["TOTAL_TIMESTEPS"] // config["NUM_STEPS"] // config["NUM_ENVS"]
)
config["MINIBATCH_SIZE"] = (
    config["NUM_ENVS"] * config["NUM_STEPS"] // config["NUM_MINIBATCHES"]
)

if(config["ENV_NAME"]=="craftax"):
    from craftax.craftax.envs.craftax_symbolic_env import CraftaxSymbolicEnvNoAutoReset
    env=CraftaxSymbolicEnvNoAutoReset()
    env_params=env.default_params
    action_dim_env = env.action_space(env_params).n
    env = LogWrapper(env)
    env = ThinkingWrapper(env, action_dim_env, config["R_THINK"])
    env = OptimisticResetVecEnvWrapper(
            env,
            num_envs=config["NUM_ENVS"],
            reset_ratio=min(16, config["NUM_ENVS"]),
        )
else:
    env, env_params = gymnax.make(config["ENV_NAME"])
    action_dim_env = env.action_space(env_params).n 
    env = FlattenObservationWrapper(env)
    env = LogWrapper(env)
    env = ThinkingWrapper(env, action_dim_env, config["R_THINK"])
    env = BatchEnvWrapper(env,config["NUM_ENVS"])


NameError: name 'config' is not defined